In [13]:
# import pandas as pd
# import pickle
# import requests
# import json
# import re

# from bs4 import BeautifulSoup

# ## url pickle load # 건축/설계 제외 우선
# pickles = ['지구한바퀴_세계여행?q=g','시사·이슈?q=g','IT_트렌드?q=g',
#                  '취향저격_영화_리뷰?q=g','오늘은_이런_책?q=g','뮤직_인사이드?q=g','직장인_현실_조언?q=g'
#                  ,'스타트업_경험담?q=g']

# writer_list = []
# for file in pickles:
#     with open(file[:-4]+"_userId.txt","rb") as fr:
#         writers = pickle.load(fr)
#     writer_list.append(writers)  ## [[카테고리1 게시글 url...],[카테고리2 게시글 url], ....[카테고리24 게시글 url]]

In [8]:
import pandas as pd
import pickle
import requests
import json
import re
from tqdm import tqdm

from bs4 import BeautifulSoup

## url pickle load
# pickles = ['IT_트렌드?q=g','취향저격_영화_리뷰?q=g','오늘은_이런_책?q=g','뮤직_인사이드?q=g','직장인_현실_조언?q=g'
#                  ,'스타트업_경험담?q=g']
pickles = ['스타트업_경험담?q=g']
# '지구한바퀴_세계여행?q=g','시사·이슈?q=g','IT_트렌드?q=g','취향저격_영화_리뷰?q=g'


writer_list = []
for file in pickles:
#     print(file)
    with open(file[:-4]+"_userId.txt","rb") as fr:
        writers = pickle.load(fr)
    writer_list.append(writers)  ## [[카테고리1 게시글 url...],[카테고리2 게시글 url], ....[카테고리24 게시글 url]]

## 게시글 속 정보 수집
def def_craw(writer):
    
    json_data = {}
    data = []
    res_text = []
    tag_keyword=[]
    
    tag_title,tag_nickname,tag_publish_date,tag_url,tag_url_plink = None,None,None,None,None
    tag_share,tag_like = str,str
    for idx,url in enumerate(writer):
        if idx % 500 == 0 : print("전체",len(writer),"중에",idx)
        if res_text == []: # 첫 시작에러 방지
            pass
        else :
            # to json
            json_data['title'] = tag_title  
            json_data['nickname'] = tag_nickname
            json_data['publish_date'] = tag_publish_date
            json_data['keyword'] = tmp_keyword   
            json_data['like'] = tag_like # like 없는 경우 ''
            json_data['share'] = tag_share # share 없는 경우 None            
            json_data['comment'] = tag_comment # comment 없는 경우 ''
            json_data['url'] = tag_url
            json_data['url_plink'] = tag_url_plink 
            json_data['text'] = res_text

        data.append(json_data)
        
        json_data = {} # 누적방지 초기화
        tmp_keyword = [] # 누적방지 초기화
        res_text = [] # 누적방지 초기화 
        
        # beautifulsoup
        html = requests.get('https://brunch.co.kr{text_url}'.format(text_url=url))
        soup = BeautifulSoup(html.text, 'html.parser')
        
        if soup.find('title').text == "brunch":
            pass
        else:
            tag_title = soup.find('title').text # 게시글 title
            tag_url = soup.find("meta",property='og:url')['content'] # 게시글 본주소
            tag_nickname = soup.find("meta",{'name':'article:media_name'})['content'] # 작가 nickname
            tag_url_plink = soup.find("meta",property='dg:plink')['content'] # 암호주소? # 모바일?
            tag_publish_date = soup.find("meta",property='article:published_time')['content'] # 발행일
            tag_keyword = soup.find_all('a',href=re.compile('/keyword')) # 게시글 키워드
            tag_like = soup.find('span',{'class':'f_l text_like_count text_default text_with_img_ico ico_likeit_like #like'}) #좋아요 수
            tag_share = soup.find('span',{'class':'f_l text_share_count text_default text_with_img_ico'}) # 공유 수
            tag_comment = soup.find('span',{'class':'f_l text_comment_count text_default text_with_img_ico'}) # 댓글 수
            text_h4 = soup.find_all(class_='wrap_item item_type_text')
            
            for text in text_h4:
                res_text.append(text.text)
    
            if tag_like == None:
                tag_like = "0"
            else:
                tag_like = tag_like.text # 좋아요 수

            if tag_share == None:
                tag_share == "0"
            else:
                tag_share = tag_share.text # 공유 수

            if tag_comment == None:
                tag_comment =="0"
            else:
                tag_comment = tag_comment.text

            for keyword in tag_keyword:
                tmp_keyword.append(keyword.text)
                
    return data ## 수집한 정보를 담은 dictionary로 반환


categories = ['스타트업_경험담']
# categories = ['지구한바퀴_세계여행,','시사_이슈','IT_트렌드','취향저격_영화리뷰','오늘은_이런책',
#              '뮤직_인사이드','직장인_현실조언','스타트업_경험담']


## 카테고리 -> 게시글의 순서로 2차 크롤링 진행
## 카테고리별로 정보를 담은 json 형식으로 저장(총 24개 json file)
from collections import OrderedDict
for idx,writer in enumerate(writer_list):
    to_json = None
    data = def_craw(writer) # 2단계 크롤링 실행
    
    del data[0]
    del data[0]
    
    to_json = OrderedDict()
    to_json['name'] = categories[idx] # category name
    to_json['version'] = "2020-05-19"
    to_json['data'] = data
    
    with open(categories[idx]+".json","w") as make_file:
        json.dump(to_json,make_file)

전체 12800 중에 0
전체 12800 중에 500
전체 12800 중에 1000
전체 12800 중에 1500
전체 12800 중에 2000
전체 12800 중에 2500
전체 12800 중에 3000
전체 12800 중에 3500
전체 12800 중에 4000
전체 12800 중에 4500
전체 12800 중에 5000
전체 12800 중에 5500
전체 12800 중에 6000
전체 12800 중에 6500
전체 12800 중에 7000
전체 12800 중에 7500
전체 12800 중에 8000
전체 12800 중에 8500
전체 12800 중에 9000
전체 12800 중에 9500
전체 12800 중에 10000
전체 12800 중에 10500
전체 12800 중에 11000
전체 12800 중에 11500
전체 12800 중에 12000
전체 12800 중에 12500
